In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import re
import sys
import time
import pymysql as mdb
import sqlalchemy

In [2]:
# list files in athlete_html
ath_htmls = [ f for f in listdir('/Users/Eddie/Desktop/athlete_html/') if isfile(join('/Users/Eddie/Desktop/athlete_html/',f)) ]
i = 0
log = pd.DataFrame() # data frame to port to SQL

# iterate over all athelete files
for ath_html in ath_htmls:
    
    # grabs user number
    is_ath = re.search(r'athlete(\d+).txt',ath_html)
    
    # determines if valid file
    if is_ath:

        # to keep track of progress
        i = i + 1
        print str(i) + ' out of ' + str(len(ath_htmls))   

        user_actual = is_ath.group(1) # gets actual user id for verification
        
        # variables to hold data
        date = ''
        activity = 0
        user = ''
        time_of_day = ''
        distance = 0
        pace = 0

        # open file and iterate through
        f = open("/Users/Eddie/Desktop/athlete_html/athlete"+str(user_actual)+".txt")
        for line in f:

            # regex to find information
            date_search = re.search(r'<time class=\"day\">(\w+), (\w.+)<\/time>',line)
            activity_search = re.search(r'<a href=\"\/activities\/(\d+)\" class=\"minimal\">',line)
            user_search = re.search(r'<a href=\"\/athletes\/(\d+)" class=\"athlete-name minimal\">',line)
            start_time_search1 = re.search(r'<time class=\"start-time\">(\w.+)<\/time>',line)
            start_time_search2 = re.search(r'<time>(\w+)<\/time>',line)        
            distance_search = re.search(r'<li title=\"Distance\">(\d.+)<span class=\"unit\">mi<\/span><\/li>',line)
            pace_search = re.search(r'<li title=\"Average Pace\">(\d+):(\d+)<span class=\"unit\">\/mi<\/span><\/li>',line)

            # identify run quantities
            if date_search:
                date = date_search.group(2)
            if activity_search:
                activity = activity_search.group(1)
            if user_search:
                user = user_search.group(1)
            if start_time_search1:
                time_of_day = start_time_search1.group(1)
            if start_time_search2:
                time_of_day = start_time_search2.group(1)            
            if distance_search:
                distance = distance_search.group(1)
            if pace_search and str(user) == user_actual:
                pace = 60*int(pace_search.group(1))+int(pace_search.group(2))
#                 print user, date, time_of_day, activity, distance, pace
                # store run in log
                log = log.append([[user,date,time_of_day,activity,distance,pace]])

1 out of 2625
2 out of 2625
3 out of 2625
4 out of 2625
5 out of 2625
6 out of 2625
7 out of 2625
8 out of 2625
9 out of 2625
10 out of 2625
11 out of 2625
12 out of 2625
13 out of 2625
14 out of 2625
15 out of 2625
16 out of 2625
17 out of 2625
18 out of 2625
19 out of 2625
20 out of 2625
21 out of 2625
22 out of 2625
23 out of 2625
24 out of 2625
25 out of 2625
26 out of 2625
27 out of 2625
28 out of 2625
29 out of 2625
30 out of 2625
31 out of 2625
32 out of 2625
33 out of 2625
34 out of 2625
35 out of 2625
36 out of 2625
37 out of 2625
38 out of 2625
39 out of 2625
40 out of 2625
41 out of 2625
42 out of 2625
43 out of 2625
44 out of 2625
45 out of 2625
46 out of 2625
47 out of 2625
48 out of 2625
49 out of 2625
50 out of 2625
51 out of 2625
52 out of 2625
53 out of 2625
54 out of 2625
55 out of 2625
56 out of 2625
57 out of 2625
58 out of 2625
59 out of 2625
60 out of 2625
61 out of 2625
62 out of 2625
63 out of 2625
64 out of 2625
65 out of 2625
66 out of 2625
67 out of 2625
68 o

In [5]:
# clean data frame
log = log.drop_duplicates()
log.columns = ['user','run_date','run_start_time','run_activity','run_distance','run_pace']
log = log.reset_index()
del log['index']

In [7]:
# store in database
con = mdb.connect('localhost','root','','runnersdb')
cur = con.cursor()
sqlEngine = sqlalchemy.create_engine('mysql+pymysql://root@localhost/runnersdb')
cur.execute("DROP TABLE IF EXISTS updated_log")
log.to_sql('updated_log',sqlEngine)